In [ ]:
#All the header files required for the code
import numpy as np
import pandas as pd
from factor_analyzer import FactorAnalyzer
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
import random

In [19]:
#Importing both the file using pandas
data1 = pd.read_csv('/content/movies data.csv')
data2 = pd.read_csv('/content/ratings data.csv')

In [20]:
#Deleting unnecessary columns
data1 = data1.drop('Unnamed: 0',axis = 1)
data2 = data2.drop(['Unnamed: 0','Timestamp'],axis = 1)

In [21]:
data1.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [22]:
data2.head()

,UserID,MovieID,Rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [23]:
#Merging both the dataframes
data = pd.merge(data2 , data1 , how='outer', on='MovieID')

In [24]:
data.head()

,UserID,MovieID,Rating,Title,Genres
0,1.0,1193,5.0,One Flew Over the Cuckoo's Nest (1975),Drama
1,2.0,1193,5.0,One Flew Over the Cuckoo's Nest (1975),Drama
2,12.0,1193,4.0,One Flew Over the Cuckoo's Nest (1975),Drama
3,15.0,1193,4.0,One Flew Over the Cuckoo's Nest (1975),Drama
4,17.0,1193,5.0,One Flew Over the Cuckoo's Nest (1975),Drama


In [25]:
# Data Processing
# Converting Genres into different columns
# Here we just create columns and put there initial value as 0
x = data.Genres
a = list()
for i in x:
    abc = i
    a.append(abc.split('|'))
a = pd.DataFrame(a)
b = a[0].unique()
for i in b:
    data[i] = 0
data.head(2)

,UserID,MovieID,Rating,Title,Genres,Drama,Animation,Musical,Action,Comedy,...,Thriller,Crime,Western,Documentary,Mystery,Horror,Sci-Fi,Film-Noir,War,Fantasy
0,1.0,1193,5.0,One Flew Over the Cuckoo's Nest (1975),Drama,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2.0,1193,5.0,One Flew Over the Cuckoo's Nest (1975),Drama,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
# we assign 1 to all the columns which are present in the Genres
for i in b:
    data.loc[data['Genres'].str.contains(i), i] = 1

In [27]:
data.head(2)

,UserID,MovieID,Rating,Title,Genres,Drama,Animation,Musical,Action,Comedy,...,Thriller,Crime,Western,Documentary,Mystery,Horror,Sci-Fi,Film-Noir,War,Fantasy
0,1.0,1193,5.0,One Flew Over the Cuckoo's Nest (1975),Drama,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2.0,1193,5.0,One Flew Over the Cuckoo's Nest (1975),Drama,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# Now there is no use of genre
# Since we have movie id so there is no need for movie names as well
data = data.drop(['Genres','Title'],axis =1)
data.head()

,UserID,MovieID,Rating,Drama,Animation,Musical,Action,Comedy,Adventure,Romance,...,Thriller,Crime,Western,Documentary,Mystery,Horror,Sci-Fi,Film-Noir,War,Fantasy
0,1.0,1193,5.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2.0,1193,5.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,12.0,1193,4.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,15.0,1193,4.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,17.0,1193,5.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
data.columns

Index(['UserID', 'MovieID', 'Rating', 'Drama', 'Animation', 'Musical',
       'Action', 'Comedy', 'Adventure', 'Romance', 'Children's', 'Thriller',
       'Crime', 'Western', 'Documentary', 'Mystery', 'Horror', 'Sci-Fi',
       'Film-Noir', 'War', 'Fantasy'],
      dtype='object')

In [30]:
# Because of merging some null values are created
data.isnull().sum()

,0
UserID,254
MovieID,0
Rating,254
Drama,0
Animation,0
Musical,0
Action,0
Comedy,0
Adventure,0
Romance,0


In [31]:
#WE simply drop the null values coz the are not treatable
data.dropna(inplace= True )

In [32]:
data.isnull().sum()

,0
UserID,0
MovieID,0
Rating,0
Drama,0
Animation,0
Musical,0
Action,0
Comedy,0
Adventure,0
Romance,0


In [33]:
#By different meathods I found 8 cluster are better
kmeanModel = KMeans(n_clusters=8)
kmeanModel.fit(data)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans()

In [34]:
# Creating an extra column in data for storing the cluster values
data['Cluster'] = kmeanModel.labels_
data['Cluster'].sample(n=10)

,Cluster
262208,2
522146,6
234778,3
406699,3
274250,6
463594,4
240709,2
370921,0
211767,0
86114,2


In [35]:
data['Cluster'].value_counts()

,count
Cluster,
6,85842
1,80269
4,71389
0,71090
2,70599
3,62516
5,61896
7,52924


In [36]:
data.head()

,UserID,MovieID,Rating,Drama,Animation,Musical,Action,Comedy,Adventure,Romance,...,Crime,Western,Documentary,Mystery,Horror,Sci-Fi,Film-Noir,War,Fantasy,Cluster
0,1.0,1193,5.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2.0,1193,5.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,12.0,1193,4.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,15.0,1193,4.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,17.0,1193,5.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [37]:
# When we merge the dataframe for a single movie multiple rows were created so a single movie is allotted
# to many clusters so here we allot a single cluster to a movie
# the Cluster which occurs maximum number of times is alloted to the movie
e = []
def fi(group):
    a = pd.DataFrame(group)
    b = pd.DataFrame(a['Cluster'].value_counts())
    d = a.index
    c = [a['MovieID'][d[0]],int(b.idxmax())]
    e.append(c)


In [38]:
data.groupby("MovieID").apply(lambda x: fi(x))

<ipython-input-37-3843108e3717>:9: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  c = [a['MovieID'][d[0]],int(b.idxmax())]


""


In [39]:
e = pd.DataFrame(e)

In [40]:
e.head()

,0,1
0,1,1
1,2,6
2,3,6
3,4,1
4,5,1


In [41]:
# I Dont know why always the column name shift according to its will :(
# Here just the column names are swapped
e.rename(columns = {0:'MovieID',1:'Cluster'},inplace=True)
e.drop_duplicates(inplace=True)

In [42]:
e.head(10)

,MovieID,Cluster
0,1,1
1,2,6
2,3,6
3,4,1
4,5,1
5,6,6
6,7,1
7,8,6
8,9,1
9,10,1


In [43]:
data1 = pd.read_csv('movies data.csv')
new_data = pd.merge(e , data1 , how='outer', on='MovieID')

In [44]:
# These were the movies we deleted while merging the file
new_data.isnull().sum()

,0
MovieID,0
Cluster,254
Unnamed: 0,0
Title,0
Genres,0


In [45]:
# We can delete the movies but I just label them randomly :)
new_data.fillna(random.randint(0,8),inplace=True)

In [46]:
new_data.isnull().sum()

,0
MovieID,0
Cluster,0
Unnamed: 0,0
Title,0
Genres,0


In [47]:
#This function select the cluster for a user according the the user choice
def select_c():
    global l
    print('Select The Movies Id you would like to watch:')
    l=[]
    for i in range(15):
        l.append(random.randint(0,3883))
    for i in l:
        print(new_data['MovieID'][i] , new_data['Title'][i],sep='--->')
    print('--------------------------------------------------------------------')
    l = int(input())
    l = new_data['Cluster'][new_data.MovieID == l]

In [48]:
# This is the main function which recommend you movies
def main():
    ans = False
    while not ans:
        select_c()
        print(new_data['Title'][new_data.Cluster == int(l)].sample(n=10))
        print('--------------------------------------------------------------------')
        print('Do you like these movies(y/n)')
        abc = input()
        while ((abc =='y') or (abc == 'Y')):
            print(new_data['Title'][new_data.Cluster == int(l)].sample(n=10))
            print('--------------------------------------------------------------------')
            print('Want more!!!!(y/n)')
            abc = input()
            if ((abc =='N') or (abc == 'n')):
                ans =True

In [49]:
main()

Select The Movies Id you would like to watch:
118--->If Lucy Fell (1996)
3487--->Dorado, El (1967)
1525--->Warriors of Virtue (1997)
3304--->Blue Collar (1978)
3893--->Nurse Betty (2000)
2708--->Autumn Tale, An (Conte d'automne) (1998)
3876--->Jerry & Tom (1998)
149--->Amateur (1994)
1445--->McHale's Navy (1997)
1330--->April Fool's Day (1986)
197--->Stars Fell on Henrietta, The (1995)
1796--->In God's Hands (1998)
1145--->Snowriders (1996)
2573--->Tango (1998)
1002--->Ed's Next Move (1996)
--------------------------------------------------------------------
1002


<ipython-input-48-0e871dbf688d>:6: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  print(new_data['Title'][new_data.Cluster == int(l)].sample(n=10))


611                 Mission: Impossible (1996)
545       Slingshot, The (Kådisbellan ) (1993)
1099                   Army of Darkness (1993)
976                    Murder, My Sweet (1944)
26                         Now and Then (1995)
431                          Free Willy (1993)
1034    Monty Python and the Holy Grail (1974)
494              Road to Wellville, The (1994)
1027               Escape from New York (1981)
875                    Of Human Bondage (1934)
Name: Title, dtype: object
--------------------------------------------------------------------
Do you like these movies(y/n)
y


<ipython-input-48-0e871dbf688d>:11: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  print(new_data['Title'][new_data.Cluster == int(l)].sample(n=10))


890                                Algiers (1938)
51                                Lamerica (1994)
596                All Dogs Go to Heaven 2 (1996)
72                            Bed of Roses (1996)
1180                 Pink Floyd - The Wall (1982)
1013                          Funeral, The (1996)
88             Journey of August King, The (1995)
1033                      Private Benjamin (1980)
339                           Forrest Gump (1994)
294     Red Firecracker, Green Firecracker (1994)
Name: Title, dtype: object
--------------------------------------------------------------------
Want more!!!!(y/n)
n
